In [5]:
# Import packages
import pandas as pd

#Load data
cars_trips = pd.read_csv("../../a_data/aa_raw_data/cars_trips.csv")
cars_country = pd.read_csv("../../a_data/aa_raw_data/cars_country.csv")
cars_fleet = pd.read_csv("../../a_data/aa_raw_data/cars_fleet.csv")
cars_price  = pd.read_csv("../../a_data/aa_raw_data/cars_price.csv")

In [ ]:
# 1. Function to get the table basic stats and metadata in a markdown table structure:
def getMetaTable(df):
    for col in df.columns:
        a = df[col].dtypes
        b = df[col].isna().sum()
        c = len(df[col].unique())
        if a=='int64' or a =='float64':
            d = round(df[col].min(),2)
            e = round(df[col].max(),2)
            f = round(df[col].mean(),2)
        else:
            d = ' -- '
            e = ' -- '
            f = ' -- ' 
        print( '| ', col,' |', a, ' | ' , b, '|' , c,' | ' ,d,' | ' ,e, ' | ', f, '|')
    return  

In [ ]:
# 1. head to understand the data available:
print(cars_trips.head())

In [ ]:
# 1. print the markdown table:
print("| Var  | Var_type | NULL_Values  | Distincts_Values | Min_Values | Max_Values | Average | ")
print("|---|---|---|---|---|---|---|")
getMetaTable(cars_trips);


In [ ]:
# 1.1. cars_fleet - check NaNs:
nan_rows  =  cars_fleet[cars_fleet['horsepower'].isnull()]
print(nan_rows)

In [19]:
# 1.1. cars_fleet - Get the cars manufacturer:
# get manufacturer by splitting the data:
cars_fleet["custom_car_manufacturer"] = cars_fleet["car_name"].str.split(" ", n = 1, expand = True)[0]

# Check the output manufacturer:
#print(cars_fleet.groupby('custom_car_manufacturer').id.nunique())

# Clean manufacturer:
cars_fleet["custom_car_manufacturer"] = cars_fleet["custom_car_manufacturer"].replace(
    [ "chevroelt", "maxda", "mercedes","toyouta", "vokswagen","vw"],
    [ "chevrolet", "mazda", "mercedes-benz", "toyota", "volkswagen","volkswagen"]
)

# save
cars_fleet.to_csv('../../a_data/ab_processed_data/p_cars_fleet.csv', index=False)  


In [ ]:
# 1.5. cars_trips - check start_time NaNs:
cars_trips['custom_end_day'] = cars_trips['end_time'].str.slice(0, 10)
cars_trips['custom_start_longitude'] = round(cars_trips['start_longitude'])
cars_trips['custom_start_latitude'] = round(cars_trips['start_latitude'])

nan_rows  =  cars_trips[cars_trips['start_time'].isnull()]

#print(cars_trips.groupby('custom_end_day').car_id.nunique())
print(nan_rows.groupby('custom_start_longitude').agg('count').car_id)
print(nan_rows.groupby('custom_start_latitude').agg('count').car_id)



In [ ]:
# 1.5. cars_trips - check latitude=0:
#print(cars_trips[cars_trips['start_latitude']==0].head())

# 1.5. cars_trips - check trip_distance NaNs:
print(cars_trips[cars_trips['trip_distance'].isnull()].head())


In [ ]:
# 1.5. cars_trips -  calculating trip_distance manually
# NOT USED IN THE FINAL DATA  - as is mentioned in the markdown document, due to quality guarantee I rather to research further before using a distance self calculation

# based on answers: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
# I simply switch math package to numpy to be able to work with pandas dataframes

import numpy as np # import sin, cos, sqrt, arctan2, radians, pi
# approximate radius of earth in km
#R = 6373.0

def calcDistance(lat1, lat2, lon1, lon2):
    R = 6373
    lat1 = radians(lat1)
    lon1 = radians(lat2)
    lat2 = radians(lon1)
    lon2 = radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * arctan2(sqrt(a), sqrt(1 - a))
    return  R * c

cars_trips["custom_trip_distance"] =calcDistance(cars_trips["start_latitude"], cars_trips["end_latitude"], cars_trips["start_longitude"], cars_trips["end_longitude"])

print(cars_trips.head())
cars_trips = cars_trips.drop('custom_trip_distance', axis=1)

In [16]:
# 1.5. cars_trips - getting trip minutes:

cars_trips['start_time'] = pd.to_datetime(cars_trips['start_time']) # transform dates to date-time format
cars_trips['end_time'] = pd.to_datetime(cars_trips['end_time'])
#get the difference in the variables
cars_trips['custom_trip_minutes'] = (cars_trips['end_time'] - cars_trips['start_time']).astype('timedelta64[s]')/60
#remove redundant variables from the trip
cars_trips = cars_trips.drop(['start_time','end_time','start_longitude','start_latitude','end_longitude','end_latitude','store_and_fwd_fl'], axis=1)
#save the data
cars_trips.to_csv('../../a_data/ab_processed_data/p_cars_trips.csv', index=False)


KeyError: 'start_time'

In [10]:
print(cars_trips.columns)

Index(['car_id', 'trip_distance', 'rate_code_id', 'custom_trip_minutes'], dtype='object')


In [17]:
cars_trips.to_csv('../../a_data/ab_processed_data/p_cars_trips.csv', index=False)